In [2]:
#VGG16 on augmented data and epochs
#used this in the report
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally
    transforms.RandomRotation(10),      # Randomly rotate the images by 10 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation and Test transform does not need augmentation, only resizing and normalization
val_test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Apply the updated transforms to datasets
train_dataset = datasets.ImageFolder(root='train', transform=train_transform)
val_dataset = datasets.ImageFolder(root='val', transform=val_test_transform)
test_dataset = datasets.ImageFolder(root='test', transform=val_test_transform)

# Create dataloaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [4]:

# Load the pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Freeze the feature layers (optional, if you want to fine-tune only the classifier)
for param in vgg16.features.parameters():
    param.requires_grad = False

# Modify the classifier for binary classification
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 2)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

# Function for training
def train_model_vgg16(model, criterion, optimizer, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}")

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}, Validation Accuracy: {accuracy:.2f}%")

        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}, Test Accuracy: {accuracy:.2f}%")


# Example of how to call the train_model_vgg16 function
# train_model_vgg16(vgg16, criterion, optimizer, train_loader, val_loader, epochs=5)

# Train the model
train_model_vgg16(vgg16, criterion, optimizer, train_loader, val_loader, epochs=20)



Epoch 1/20, Training Loss: 0.4413922432262988
Epoch 1, Validation Accuracy: 93.75%
Epoch 1, Test Accuracy: 88.14%
Epoch 2/20, Training Loss: 0.24962903178983334
Epoch 2, Validation Accuracy: 100.00%
Epoch 2, Test Accuracy: 83.49%
Epoch 3/20, Training Loss: 0.2313819032703795
Epoch 3, Validation Accuracy: 100.00%
Epoch 3, Test Accuracy: 86.86%
Epoch 4/20, Training Loss: 0.15431607505344097
Epoch 4, Validation Accuracy: 100.00%
Epoch 4, Test Accuracy: 81.41%
Epoch 5/20, Training Loss: 0.20874274484088295
Epoch 5, Validation Accuracy: 100.00%
Epoch 5, Test Accuracy: 84.94%
Epoch 6/20, Training Loss: 0.13106373870624602
Epoch 6, Validation Accuracy: 100.00%
Epoch 6, Test Accuracy: 85.58%
Epoch 7/20, Training Loss: 0.13259256605888814
Epoch 7, Validation Accuracy: 87.50%
Epoch 7, Test Accuracy: 82.05%
Epoch 8/20, Training Loss: 0.11334933135644218
Epoch 8, Validation Accuracy: 100.00%
Epoch 8, Test Accuracy: 88.62%
Epoch 9/20, Training Loss: 0.15282195849426983
Epoch 9, Validation Accuracy: